## Description

All functions for interfacing the course_schedule database in various ways.

@author Gaurav Kuwar, Author B, Author C

@version 1.00 May, 2022


### Import Modules

In [16]:
# if the module is not installed yet, we pip install it
# pip install mysql-connector-python
import mysql.connector

import numpy as np
import pandas as pd

### Get config Info

In [90]:
def getConfig(authFn='auth.txt'):
    """
        Reads and formats data from authFn returns it.
    """
    # Read the authentication file
    auth = np.loadtxt('auth.txt', dtype = str)
    # Config dictionary
    labels = ['username', 'password', 'host', 'database']
    config = dict(zip(labels, list(auth)))
    return config

config = getConfig()

## All Modules

List of functions we need:

- populateRelation(relation, csvFile) - func to populate db from csv (prop just need 1)
- selectAll(relation) - a func to SELECT * FROM table for a table

- Professor
    - SELECT
        - selectAll("Professor") but to get list of all prof
    - INSERT
        - addNewProf - insert a new prof record
    - DELETE
        - deleteProf delete a prof record (by professor_id)
    - UPDATE
        - updateProfInfo - change prof info (by professor_id)
- Room
    - SELECT
        - roomsInBuilding(building_name) - get all rooms in a building
        - allBuildings() - get list of all buildings
    - INSERT
        - add new room record
    - DELETE
        - remove a room record
    - UPDATE
        - change room info (based on room_id?)
- Course
    - SELECT
        - selectAll("Courses") - get list of all the courses
        - get list of all the courses by department
    - INSERT
        - insert new course
    - DELETE
        - delete a course (based on course_num and department or based on id)
    - UPDATE
        - update a course  based on id
- Class
    - SELECT
        - selectAll("Classes") get list of all classes
        - get list based on year
        - get list based on year and term
        - get list based on course_id
        - get list based on course_id, year, term (all the sections of a course for a semester"
    - INSERT
        - insert a new class record
        - insert a new class record with section_id, room_id, year, term, professor_id, course_num, department, start_time, end_time and days_of_the_week.
    - DELETE
        - delete a class record based on section_id, course_id, year, term
    - UPDATE
        - update a class record based on section_id, course_id, year, term

## Select all from Relation

In [81]:
def selectAll(relation, config=config):
    """
        Takes a relation which the name of a relation in the database in config
        and returns all rows in the relation.
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    # SELECT query
    query = ("SELECT * FROM `{relation}`".format(relation=relation))
    # Run the query
    cursor.execute(query)
    col_names = cursor.column_names # get column names
    # Fetch the results into a list
    records = [record for record in cursor]
    
    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return pd.DataFrame(records, columns=col_names) # return as a pandas Dataframe (looks nicer)

In [77]:
def allProfessors(config=config):
    """
        Returns list of all professors in Professor relation and their info.
    """
    return selectAll("Professor", config=config)

In [78]:
def allCourses(config=config):
    """
        Returns list of all courses in Course relation and their info.
    """
    return selectAll("Course", config=config)

In [79]:
def allRooms(config=config):
    """
        Returns list of all rooms in Room relation and their info.
    """
    return selectAll("Room", config=config)

In [80]:
def allClasses(config=config):
    """
        Returns list of all classes in Class relation and their info.
    """
    return selectAll("Class", config=config)